In [0]:
%pip install "mlflow>=1.25.0"
%pip install keras
%pip install tensorflow

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import mlflow.pyfunc
from mlflow.models.signature import infer_signature
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np

In [0]:

# Code này đã bao gồm TẤT CẢ các import, class, và logic

# === CELL 1: IMPORTS ===
import mlflow
import mlflow.pyfunc  
import pandas as pd
import numpy as np
from pyspark.sql.functions import col, when
import datetime

# === CELL 2: ĐỊNH NGHĨA CLASS (PHẢI CÓ) ===
# (Class này phải được định nghĩa để mlflow.load_model() hoạt động)

class RainPredictionWrapper(mlflow.pyfunc.PythonModel):

    def __init__(self, model, scaler, seq_len, 
                 feature_columns, numeric_feature_cols, location_cols):
        self.model = model
        self.scaler = scaler
        self.seq_len = seq_len
        self.feature_columns = feature_columns
        self.numeric_feature_cols = numeric_feature_cols
        self.location_cols = location_cols
        self.n_features = len(self.feature_columns)

    def _preprocess(self, model_input_df):
        # 1. Tách riêng 2 nhóm cột (giống hệt lúc training)
        numeric_data = model_input_df[self.numeric_feature_cols]
        location_data = model_input_df[self.location_cols].values
        
        # 2. CHỈ scale các cột numeric (đã được lag)
        scaled_numeric = self.scaler.transform(numeric_data)
        
        # 3. Kết hợp lại
        scaled_data = np.hstack([scaled_numeric, location_data])
        
        # 4. Reshape cho LSTM [samples, timesteps, features]
        input_sequence = scaled_data.reshape(1, self.seq_len, self.n_features)
        return input_sequence

    def predict(self, context, model_input):
        if model_input.shape[0] != self.seq_len:
            raise ValueError(f"Input data must have exactly {self.seq_len} rows (timesteps).")
            
        input_sequence = self._preprocess(model_input)
        prediction_prob = self.model.predict(input_sequence) # Shape (1, 1)
        
        prob = float(prediction_prob[0][0])
        label = 1 if prob > 0.5 else 0
        
        return pd.DataFrame({
            "prediction_label": [label],
            "prediction_probability": [prob]
        })

print("--- Cell 1 & 2: Imports và Class Definition đã sẵn sàng ---")


# === CELL 3: THIẾT LẬP CẤU HÌNH PIPELINE ===
print("\n--- Cell 3: Thiết lập Cấu hình Pipeline ---")

UC_CATALOG_NAME = "hcmut"
UC_SCHEMA_NAME = "gold"
MODEL_NAME = "weatherforecast_rain_lstm" 
MODEL_ALIAS = "rain"                   
MODEL_REGISTRY_NAME = f"{UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.{MODEL_NAME}"

SOURCE_TABLE = "hcmut.gold.fact_vn_weather_daily" 
TARGET_TABLE = "hcmut.gold.lstm_rain_daily"

CITIES_TO_FORECAST = ["Ho Chi Minh City", "Da Nang City", "Ha Noi City"]
CITY_LOC_COLS = {"Ho Chi Minh City": "loc_hcm", "Da Nang City": "loc_dn", "Ha Noi City": "loc_hn"}

N_LAG_DAYS = 24 
SEQ_LEN = 3     

numeric_cols = [
    "nr_temperature_2m_mean", "nr_relative_humidity_2m_mean", "nr_precipitation_sum",
    "nr_pressure_msl_mean", "nr_cloud_cover_mean", "nr_sunshine_duration",
    "nr_wind_speed_10m_mean", "nr_soil_moisture_0_to_7cm_mean"
]
location_cols = ["loc_hcm", "loc_dn", "loc_hn"]
BASE_COLUMNS_TO_SELECT = ["dt_date_record", "ds_location"] + numeric_cols

# === CELL 4: TẢI MÔ HÌNH (PHỤ THUỘC CELL 1 & 2) ===
print("\n--- Cell 4: Tải Mô hình từ Registry ---")

mlflow.set_registry_uri("databricks-uc")

print(f"Đang tải mô hình '{MODEL_REGISTRY_NAME}' với bí danh (alias) '@{MODEL_ALIAS}'...")
model_uri = f"models:/{MODEL_REGISTRY_NAME}@{MODEL_ALIAS}"

try:
    loaded_model = mlflow.pyfunc.load_model(model_uri)
    print("--- Tải mô hình thành công ---")
except Exception as e:
    print(f"LỖI: Không thể tải mô hình. Tên model hoặc alias không đúng.")
    print(f"Model: {MODEL_REGISTRY_NAME}, Alias: @{MODEL_ALIAS}")
    print(f"Chi tiết lỗi: {e}")

# === CELL 5: LẤY DỮ LIỆU VÀ FEATURE ENGINEERING ===
print("\n--- Cell 5: Lấy dữ liệu và Feature Engineering ---")

days_to_fetch = N_LAG_DAYS + SEQ_LEN + 10 

print(f"Đang lấy {days_to_fetch} ngày dữ liệu mới nhất từ {SOURCE_TABLE}...")
pdf = (
    spark.read.table(SOURCE_TABLE)
    .select(*BASE_COLUMNS_TO_SELECT)
    .orderBy(col("dt_date_record").desc())
    .limit(days_to_fetch * 3)
    .orderBy(col("dt_date_record").asc())
    .toPandas()
)

pdf["loc_hcm"] = (pdf["ds_location"] == "Ho Chi Minh City").astype(int)
pdf["loc_dn"] = (pdf["ds_location"] == "Da Nang City").astype(int)
pdf["loc_hn"] = (pdf["ds_location"] == "Ha Noi City").astype(int)

print(f"Đang tạo {N_LAG_DAYS} ngày lags...")
lag_dfs = []
for lag in range(1, N_LAG_DAYS + 1):
    lag_df = pdf.groupby("ds_location")[numeric_cols].shift(lag)
    lag_df.columns = [f"{c}_lag_{lag}" for c in numeric_cols]
    lag_dfs.append(lag_df)

pdf = pd.concat([pdf] + lag_dfs, axis=1)
pdf = pdf.dropna().reset_index(drop=True)

numeric_feature_cols = [c for c in pdf.columns if any(x in c for x in numeric_cols)]
feature_cols = numeric_feature_cols + location_cols

print(f"Đã tạo xong {len(feature_cols)} features (bao gồm 8 cột gốc + 192 cột lag + 3 cột location).")

# === CELL 6: DỰ BÁO VÀ LƯU KẾT QUẢ ===
print("\n--- Cell 6: Thực hiện Dự báo và Lưu kết quả ---")

all_forecasts = []
today_run_time = datetime.datetime.now(datetime.timezone.utc)
forecast_date = pdf["dt_date_record"].max() + pd.Timedelta(days=1)

for city_name, loc_col in CITY_LOC_COLS.items():
    print(f"\n--- Đang xử lý cho: {city_name} ---")
    
    try:
        city_pdf = pdf[pdf["ds_location"] == city_name].copy()
        input_data = city_pdf.tail(SEQ_LEN)[feature_cols]
        
        if input_data.shape[0] != SEQ_LEN:
            print(f"LỖI: Không đủ dữ liệu (cần {SEQ_LEN} ngày) để dự báo cho {city_name}. Bỏ qua...")
            continue
        
        try:
            for col_name in location_cols:
                input_data[col_name] = input_data[col_name].astype(np.int32)
            for col_name in numeric_feature_cols:
                input_data[col_name] = input_data[col_name].astype(np.float32)
        except Exception as e:
            print(f"LỖI khi ép kiểu dữ liệu: {e}")
            continue

        print(f"Đang dự báo cho {city_name}...")
        forecast_result_df = loaded_model.predict(input_data)
        
        forecast_result_df['dt_forecast_date'] = forecast_date
        forecast_result_df['ds_location'] = city_name
        forecast_result_df['dt_model_run_time'] = today_run_time
        forecast_result_df['ds_model_version_uri'] = model_uri
        
        all_forecasts.append(forecast_result_df)

    except Exception as e:
        print(f"LỖI trong quá trình xử lý cho {city_name}: {e}")

if not all_forecasts:
    print("\nKhông có kết quả dự báo nào được tạo ra. Kết thúc.")
else:
    print(f"\n--- TỔNG HỢP KẾT QUẢ DỰ BÁO ({len(all_forecasts)} thành phố) ---")
    final_results_df = pd.concat(all_forecasts)
    display(final_results_df)

    print(f"Đang lưu tất cả kết quả vào bảng {TARGET_TABLE}...")
    
    try:
        results_spark_df = spark.createDataFrame(final_results_df)
        (results_spark_df.write
            .format("delta")
            .mode("append")
            .option("mergeSchema", "true") 
            .saveAsTable(TARGET_TABLE)
        )
        print("--- LƯU KẾT QUẢ VÀO DELTA LAKE THÀNH CÔNG ---")
    except Exception as e:
        print(f"LỖI khi lưu vào Delta Lake: {e}")

    print(f"Đang lưu tất cả kết quả vào bảng {TARGET_TABLE}...")
    
    try:
        results_spark_df = spark.createDataFrame(final_results_df)
        (results_spark_df.write
            .format("delta")
            .mode("append")
            .option("mergeSchema", "true") 
            .saveAsTable(TARGET_TABLE)
        )
        print("--- LƯU KẾT QUẢ VÀO DELTA LAKE THÀNH CÔNG ---")
    except Exception as e:
        print(f"LỖI khi lưu vào Delta Lake: {e}")